In [3]:
# Apparantly, this is run on my laptop with Intel Grpahics Card. Pytorch will not read other than NVIDIA GPU. So the CPU will
# used instead:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle
device = 'cuda' if torch.cuda.is_available() else 'cpu'
block_size = 128
batch_size = 64
max_iters = 1000
learning_rate = 3e-3
eval_iters = 100
n_embed = 384 # embedding dimension
n_head = 4 # number of heads we can use
n_layer = 4 # number of decoder blocks
dropout = 0.2 # 20% of our neurons turn to 0
# print(device)

In [4]:
# Experimenting with the text characters:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [5]:
# Time to Tokenize:
string_to_int = {ch:i for i,ch in enumerate (chars)}
int_to_string = {i:ch for i,ch in enumerate (chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encode_hello = encode('hello')
decode_hello = decode(encode_hello)
print(encode_hello)
print(decode_hello)

[61, 58, 65, 65, 68]
hello


In [6]:
# Using Tensors instead of Array:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [7]:
n = int(0.9 * len(data))
traindata = data[:n]
valdata = data[n:]

def get_batch(split):
    data = traindata if split == 'train' else valdata
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [8]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias = False)
        self.query = nn.Linear(n_embed, head_size, bias = False)
        self.value = nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x) #B,T,hs
        q = self.query(x) #B,T,hs
        # Computing Attention Scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) @ (B,hs,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        # Performing Weighted Aggregation of values
        v = self.value(x) #(B,T,hs)
        out = wei @ v #(B,T,T) @ (B,T,hs)
        return out

class MultiHeadAttention(nn.Module): # multiple heads of self-attention in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads, n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #(B,T,F)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module): # Simple Linear Layer followed by Non-Linearity
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_embed, 4*n_embed), nn.ReLU(), nn.Linear(4*n_embed, n_embed), nn.Dropout(dropout),)
        
    def forward(self,x):
        return self.net(x)

class Block(nn.Module): # Transformer Block Followed by computation
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x
        
class GPTLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
        self.position_embedding_table = nn.Embedding(block_size,n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self,module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            
        
    def forward(self,index,targets=None):
        B,T = index.shape
        token_embed = self.token_embedding_table(index) # (B,T,C)
        position_embed = self.position_embedding_table(torch.arange(T, device=device))
        x = token_embed + position_embed #(B,T,C) Shape
        x = self.blocks(x) #(B,T,C) Shape
        x = self.ln_f(x) #(B,T,C) Shape
        logits = self.lm_head(x) #(B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits,loss

    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            # Getting Predictions
            logits,loss = self.forward(index)
            # focusing only on the last time step
            logits = logits[:,-1,:] # Becomes (B,C)
            # applying softmax to get probabilities
            probs = F.softmax(logits,dim=-1) # (B,C)
            # Sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1) #(B,1)
            # append sampled index to the running sequence
            index = torch.cat((index,index_next), dim=1) #(B,T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# A Pytorch Optimizer:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

step: 0, train loss: 4.430, val loss: 4.438
step: 100, train loss: 3.137, val loss: 3.175
step: 200, train loss: 3.146, val loss: 3.172
step: 300, train loss: 3.142, val loss: 3.171
step: 400, train loss: 3.142, val loss: 3.174
step: 500, train loss: 3.147, val loss: 3.177
step: 600, train loss: 3.141, val loss: 3.173
step: 700, train loss: 3.142, val loss: 3.176
step: 800, train loss: 3.144, val loss: 3.171
step: 900, train loss: 3.147, val loss: 3.172
3.090444326400757
model saved


In [11]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me?Ku anendobs rirf h   "ynv a o  f  ure
iuo
s oo"s huaehcuf erefi m, cuat o t " en
r 
tat!h idt uhcns'
